<a href="https://colab.research.google.com/github/ailuj/BayesianDeepLearning/blob/playground/Paper_all_results_det_simple.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:

from google.colab import drive
drive.mount('/content/drive',force_remount=True)


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!ls "/content/drive/My Drive/data_sets"


test_set_40k.csv    train_set_150k.csv	train_set_240k.csv
train_set_100k.csv  train_set_200k.csv	train_set_60k.csv


In [0]:
import pandas as pd
test_40k = pd.read_csv('/content/drive/My Drive/data_sets/test_set_40k.csv')
train_60k = pd.read_csv('/content/drive/My Drive/data_sets/train_set_60k.csv')
train_100k = pd.read_csv('/content/drive/My Drive/data_sets/train_set_100k.csv')
train_150k = pd.read_csv('/content/drive/My Drive/data_sets/train_set_150k.csv')
train_200k = pd.read_csv('/content/drive/My Drive/data_sets/train_set_200k.csv')
train_240k = pd.read_csv('/content/drive/My Drive/data_sets/train_set_240k.csv')

In [0]:
full = train_240k.append(test_40k)

In [0]:
full = full[['int_rate','loan_amnt','purpose', 'profit_rate']]
full_features = full[['int_rate','loan_amnt','purpose']]
full_features = pd.get_dummies(full_features)
full_target = full[['profit_rate']]

In [0]:
def preprocess(x):
  from sklearn import preprocessing
  x = x.dropna()
 
  x_features = x[['int_rate','loan_amnt','purpose']]
  x_features = pd.get_dummies(x_features)
  x_target = x[['profit_rate']]

  scaler_features = preprocessing.MinMaxScaler().fit(full_features)
  x_features = scaler_features.transform(x_features)
  
  scaler_target = preprocessing.MinMaxScaler().fit(full_target)
  x_target = scaler_target.transform(x_target)
  
  return (x_features, x_target)

In [0]:
((test_40k_features,test_40k_target),
 (train_60k_features,train_60k_target),
 (train_100k_features,train_100k_target),
 (train_150k_features,train_150k_target),
 (train_200k_features,train_200k_target),
 (train_240k_features,train_240k_target)) = (preprocess(test_40k), 
                                             preprocess(train_60k), 
                                             preprocess(train_100k), 
                                             preprocess(train_150k), 
                                             preprocess(train_200k), 
                                             preprocess(train_240k)) 

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype u

In [0]:
((test_40k_features.shape,test_40k_target.shape),
 (train_60k_features.shape,train_60k_target.shape),
 (train_100k_features.shape,train_100k_target.shape),
 (train_150k_features.shape,train_150k_target.shape),
 (train_200k_features.shape,train_200k_target.shape),
 (train_240k_features.shape,train_240k_target.shape))

(((40000, 16), (40000, 1)),
 ((65600, 16), (65600, 1)),
 ((105600, 16), (105600, 1)),
 ((155600, 16), (155600, 1)),
 ((205600, 16), (205600, 1)),
 ((245600, 16), (245600, 1)))

# Modeling


In [0]:
from keras import Model as mcModel
from keras import Input as mcInput
from keras.models import Sequential
from keras.layers import Dropout
from keras.layers import Dense
from keras.regularizers import l2

def architecture(model_type, h_cells, n_hidden, input_dim, dropout_prob, reg):
  
  if model_type == 'deterministic_simple':
    from keras.models import Sequential as seq1
    from keras.layers import Dropout as drop1
    from keras.layers import Dense as dense1
    
    model1 = seq1()
    model1.add(dense1(h_cells, input_dim = input_dim, activation='relu'))
    model1.add(dense1(h_cells, activation='relu'))
    model1.add(dense1(1))
    return model1
  

def model_runner(X_train, y_train, X_test=test_40k_features, y_test=test_40k_target,
                dropout_prob=0.20, n_epochs=100, tau=1.0, batch_size=500, 
                lengthscale=1e-2, n_hidden=[100,100], h_cells=100):
  
  input_dim = X_train.shape[1]
  N = X_train.shape[0]
  reg = lengthscale**2 * (1 - dropout_prob) / (2. * N * tau)

  print('Simple Deterministic NN fit')
  model_det_simple = architecture(model_type = 'deterministic_simple', 
                                  h_cells= h_cells, n_hidden=n_hidden, input_dim=input_dim, 
                                  dropout_prob=dropout_prob, reg=reg)
  
  model_det_simple.compile(optimizer='adam', loss='mse', metrics=['mae'])
  model_det_simple.fit(X_train, y_train, batch_size=batch_size, nb_epoch=n_epochs, verbose=1)
  
  
  
  return (model_det_simple)

def predictor(model_det_simple, 
              X_test=test_40k_features, y_test = test_40k_target, T = 1000):
  from sklearn.metrics import mean_squared_error
  import numpy as np
  
  pred_det_simple = model_det_simple.predict(X_test, verbose=1)
  mse_det_simple = mean_squared_error(pred_det_simple, y_test)
  print(mse_det_simple)
  
  
  
  return (pred_det_simple)

In [0]:
model_det_simple_60k = model_runner(train_60k_features, train_60k_target)
pred_det_simple_60k = predictor(model_det_simple_60k)

Simple Deterministic NN fit


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:36: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Epoch 1/100
65600/65600 [==============================] - 1s 14us/step - loss: 0.0570 - mean_absolute_error: 0.1857
Epoch 2/100
65600/65600 [==============================] - 1s 9us/step - loss: 0.0477 - mean_absolute_error: 0.1713
Epoch 3/100
65600/65600 [==============================] - 1s 9us/step - loss: 0.0477 - mean_absolute_error: 0.1712
Epoch 4/100
65600/65600 [==============================] - 1s 9us/step - loss: 0.0477 - mean_absolute_error: 0.1714
Epoch 5/100
65600/65600 [==============================] - 1s 9us/step - loss: 0.0476 - mean_absolute_error: 0.1709
Epoch 6/100
65600/65600 [==============================] - 1s 9us/step - loss: 0.0477 - mean_absolute_error: 0.1711
Epoch 7/100
65600/65600 [==============================] - 1s 9us/step - loss: 0.0476 - mean_absolute_error: 0.1709
Epoch 8/100
65600/65600 [==============================] - 1s 9us/step - loss: 0.0476 - mean_absolute_error: 0.1710
Epoch 9/100
65600/65600 [==============================] - 1s 9us/step 

In [0]:
import pickle
outfile = open('pred_det_simple_60k.pickle','wb')
pickle.dump(pred_det_simple_60k,outfile)
outfile.close()

from google.colab import files

files.download('pred_det_simple_60k.pickle')


In [0]:
model_det_simple_100k = model_runner(train_100k_features, train_100k_target)
pred_det_simple_100k = predictor(model_det_simple_100k)

Simple Deterministic NN fit


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:36: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Epoch 1/100
105600/105600 [==============================] - 1s 13us/step - loss: 0.0521 - mean_absolute_error: 0.1783
Epoch 2/100
105600/105600 [==============================] - 1s 9us/step - loss: 0.0475 - mean_absolute_error: 0.1709
Epoch 3/100
105600/105600 [==============================] - 1s 9us/step - loss: 0.0475 - mean_absolute_error: 0.1708
Epoch 4/100
105600/105600 [==============================] - 1s 9us/step - loss: 0.0474 - mean_absolute_error: 0.1706
Epoch 5/100
105600/105600 [==============================] - 1s 9us/step - loss: 0.0474 - mean_absolute_error: 0.1706
Epoch 6/100
105600/105600 [==============================] - 1s 9us/step - loss: 0.0475 - mean_absolute_error: 0.1707
Epoch 7/100
105600/105600 [==============================] - 1s 9us/step - loss: 0.0475 - mean_absolute_error: 0.1707
Epoch 8/100
105600/105600 [==============================] - 1s 9us/step - loss: 0.0475 - mean_absolute_error: 0.1706
Epoch 9/100
105600/105600 [============================

In [0]:
import pickle
outfile = open('pred_det_simple_100k.pickle','wb')
pickle.dump(pred_det_simple_100k,outfile)
outfile.close()



from google.colab import files

files.download('pred_det_simple_100k.pickle')


In [0]:
model_det_simple_150k= model_runner(train_150k_features, train_150k_target)
pred_det_simple_150k = predictor(model_det_simple_150k)

Simple Deterministic NN fit


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:36: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Epoch 1/100
155600/155600 [==============================] - 2s 12us/step - loss: 0.0529 - mean_absolute_error: 0.1792
Epoch 2/100
155600/155600 [==============================] - 1s 9us/step - loss: 0.0477 - mean_absolute_error: 0.1714
Epoch 3/100
155600/155600 [==============================] - 1s 9us/step - loss: 0.0476 - mean_absolute_error: 0.1713
Epoch 4/100
155600/155600 [==============================] - 1s 9us/step - loss: 0.0476 - mean_absolute_error: 0.1712
Epoch 5/100
155600/155600 [==============================] - 1s 9us/step - loss: 0.0476 - mean_absolute_error: 0.1712
Epoch 6/100
155600/155600 [==============================] - 1s 9us/step - loss: 0.0477 - mean_absolute_error: 0.1714
Epoch 7/100
155600/155600 [==============================] - 1s 9us/step - loss: 0.0477 - mean_absolute_error: 0.1713
Epoch 8/100
155600/155600 [==============================] - 1s 9us/step - loss: 0.0476 - mean_absolute_error: 0.1712
Epoch 9/100
155600/155600 [============================

In [0]:
import pickle
outfile = open('pred_det_simple_150k.pickle','wb')
pickle.dump(pred_det_simple_150k,outfile)
outfile.close()


from google.colab import files

files.download('pred_det_simple_150k.pickle')


In [0]:
model_det_simple_200k = model_runner(train_200k_features, train_200k_target)
pred_det_simple_200k = predictor(model_det_simple_200k)

Simple Deterministic NN fit


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:36: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Epoch 1/100
205600/205600 [==============================] - 2s 11us/step - loss: 0.0498 - mean_absolute_error: 0.1747
Epoch 2/100
205600/205600 [==============================] - 2s 9us/step - loss: 0.0476 - mean_absolute_error: 0.1713
Epoch 3/100
205600/205600 [==============================] - 2s 9us/step - loss: 0.0476 - mean_absolute_error: 0.1710
Epoch 4/100
205600/205600 [==============================] - 2s 9us/step - loss: 0.0475 - mean_absolute_error: 0.1711
Epoch 5/100
205600/205600 [==============================] - 2s 9us/step - loss: 0.0476 - mean_absolute_error: 0.1711
Epoch 6/100
205600/205600 [==============================] - 2s 9us/step - loss: 0.0476 - mean_absolute_error: 0.1711
Epoch 7/100
205600/205600 [==============================] - 2s 9us/step - loss: 0.0475 - mean_absolute_error: 0.1709
Epoch 8/100
205600/205600 [==============================] - 2s 9us/step - loss: 0.0475 - mean_absolute_error: 0.1711
Epoch 9/100
205600/205600 [============================

In [0]:
import pickle
outfile = open('pred_det_simple_200k.pickle','wb')
pickle.dump(pred_det_simple_200k,outfile)
outfile.close()


from google.colab import files


files.download('pred_det_simple_200k.pickle')


In [0]:
model_det_simple_240k = model_runner(train_240k_features, train_240k_target)
pred_det_simple_240k = predictor(model_det_simple_240k)

Simple Deterministic NN fit


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:36: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Epoch 1/100
245600/245600 [==============================] - 3s 12us/step - loss: 0.0494 - mean_absolute_error: 0.1739
Epoch 2/100
245600/245600 [==============================] - 2s 10us/step - loss: 0.0475 - mean_absolute_error: 0.1708
Epoch 3/100
245600/245600 [==============================] - 2s 10us/step - loss: 0.0475 - mean_absolute_error: 0.1707
Epoch 4/100
245600/245600 [==============================] - 2s 9us/step - loss: 0.0475 - mean_absolute_error: 0.1708
Epoch 5/100
245600/245600 [==============================] - 2s 9us/step - loss: 0.0475 - mean_absolute_error: 0.1707
Epoch 6/100
245600/245600 [==============================] - 2s 9us/step - loss: 0.0474 - mean_absolute_error: 0.1706
Epoch 7/100
245600/245600 [==============================] - 2s 9us/step - loss: 0.0474 - mean_absolute_error: 0.1707
Epoch 8/100
245600/245600 [==============================] - 2s 9us/step - loss: 0.0474 - mean_absolute_error: 0.1707
Epoch 9/100
245600/245600 [==========================

In [0]:
import pickle
outfile = open('pred_det_simple_240k.pickle','wb')
pickle.dump(pred_det_simple_240k,outfile)
outfile.close()


from google.colab import files

files.download('pred_det_simple_240k.pickle')
